In [1]:
from kafka import KafkaAdminClient
from kafka.admin import NewTopic
# Example usage
topic_name = 'output-topic'
bootstrap_servers = ["10.237.96.122:9092"]
N_PARTITIONS = 1
def delete_kafka_topic(topic_name, bootstrap_servers=bootstrap_servers):
    # Create an admin client
    admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers[0])

    # Delete the topic
    admin_client.delete_topics([topic_name])
    print(f"Topic '{topic_name}' deleted successfully.")


try:
    delete_kafka_topic(topic_name)
except Exception as e:
    print(f"Error: {e}")

# create topic foobar with N partitions
admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)
admin_client.create_topics([NewTopic(name=topic_name, num_partitions=N_PARTITIONS, replication_factor=1)])


# get number of partitions
admin_client.describe_topics(topic_name)

Topic 'user_behavior' deleted successfully.


TopicAlreadyExistsError: [Error 36] TopicAlreadyExistsError: Request 'CreateTopicsRequest_v3(create_topic_requests=[(topic='user_behavior', num_partitions=1, replication_factor=1, replica_assignment=[], configs=[])], timeout=30000, validate_only=False)' failed with response 'CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='user_behavior', error_code=36, error_message="Topic 'user_behavior' already exists.")])'.

In [2]:
import time
import pandas as pd
import json
from kafka import KafkaConsumer

# Create a KafkaConsumer instance
consumer = KafkaConsumer(
    # bootstrap_servers=['localhost:9092'],
    bootstrap_servers=["10.237.96.122:9092"],
    auto_offset_reset='latest',
    # auto_offset_reset='earliest',
    # receive_buffer_bytes=160*1024,
    enable_auto_commit=False,
    # value_deserializer=lambda m: json.loads(m.decode('ascii')),
)

# Subscribe to a specific topic
# TOPIC_NAME = "input-topic"
TOPIC_NAME = "output-topic"
consumer.subscribe(topics=[TOPIC_NAME])
N_sample =  - 1
while True:
    msg_poll = consumer.poll(timeout_ms=0)
    for k, v in msg_poll.items():
        print(k, v)
    # break

# df_record = pd.DataFrame(lst_record)
# df_record.describe()
msg_poll = consumer.poll(timeout_ms=0)
for k, v in msg_poll.items():
    print(k, v)

TopicPartition(topic='input-topic', partition=0) [ConsumerRecord(topic='input-topic', partition=0, offset=2, timestamp=1732445632071, timestamp_type=0, key=None, value=b"# Setup\n```sh\nwget https://raw.githubusercontent.com/confluentinc/cp-all-in-one/refs/heads/7.5.0-post/cp-all-in-one-flink/docker-compose.yml\ndocker compose up -d\ndocker exec -it flink-sql-client sql-client.sh\n```\n\n# Tunel\n```sh\n# list topic\ndocker exec -it broker kafka-topics --bootstrap-server broker:29092 --list\n```\n# Usage\n## create topic\n```sql\nDROP TABLE IF EXISTS KafkaTable;\nCREATE TABLE KafkaTable (\n  `user_id` BIGINT,\n  `item_id` BIGINT,\n  `behavior` STRING,\n  `ts` TIMESTAMP_LTZ(3) METADATA FROM 'timestamp'\n) WITH (\n  'connector' = 'kafka',\n  'topic' = 'user_behavior',\n  'properties.bootstrap.servers' = 'localhost:9092',\n  'properties.group.id' = 'testGroup',\n  'scan.startup.mode' = 'earliest-offset',\n  'format' = 'csv'\n);\nSELECT * FROM KafkaTable;\n```\n\n## Upsert\n```sql\nCREATE 